In [ ]:
#Generator and Discriminator
#Generator makes fakes and Discriminator decides if real or fake

#Building the discriminator
#Discriminator is 1 for black and 0 for white in a 2x2 image. 
#The difference between the Discriminator values and the noise values is the threshold.
#Top left and bottom right are positive, the rest are negative.
#Perfect would be top left / bottom right 1, the rest 0
#Example [Discriminator Total] 1*1 + 0*(-1) + 0*(-1) + 1*1 = 2
#[Noise Total] .25*1 + 1 * (-1) + .5 *(-1) + .75 * 1 = -.5
#The bias is 1 (subtracted) so Discriminator total is 1 and Noise total is -1.5
#If the threshold is more than 1, then it is real.
#Sigmoid of 1 is .73 so since 73% chance that it is real is above 50% it's considered real.
#Sigmoid of -1.5 is .18 so 18% chance it is real is less than 50% consider it a fake.

# Building a generator
# In generators we want the bias to be high in top left and bottom right (+1) and low in the others (-1)
# Random numbers between 0 and 1 are assigned to all 4 cells but the bias is added.
# Example values 1.7, -1.7, 1.7, -1.7 and the sigmoids of these values show .85 and .15
# The generator has to learn to apply this bias to get a desired result
# Error functions - different error functions can be applied this example uses log loss error function

# If label is 1 (desired result) and prediction is .1 that's a large error.
# If label is 1 (desired result) and prediction is .9 that's a small error.
# Negative log of the prediction number is the error. -ln(.1) = 2.3 and -ln(.9) = .1

# If label is 0 (non-desired result) and prediction is .9 that's a large error.
# If label is 0 (non-desired result) and prediction is .1 that's a small error.
# Negative log of (1 - prediction number) is the error. -ln( 1 - .9) = 2.3 and -ln(1 - .9) = .1

#Backpropagation - reduces the error amount as it progresses through the model.

#Whole process is the generator creates noise and creates and image then passes it to the discriminator which 
#calculates a sigmoid and checks if real or fake.
#The generators goal is to be closer to 1 using -ln(.68) and the discriminator's goal is being closer to 0 with -ln(1 - .68).
#The error functions are updated exclusive to the 2 models because of this making both better.

In [8]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

In [11]:
#We want to normalize the 3 RGB values to a single value between -1 and 1 to increase computing time.
#stats = this is the mean (0.5, 0.5, 0.5), this is the standard deviation(0.5, 0.5, 0.5)

#There's around 5000 images.
#The batch size are the number of samples that will be propagated through the network before updating the model parameters.
#Consider RAM/GPU limitations with the pixel size.
#Larger batch sizes result in faster progress in training, but don't always converge as fast.
#Smaller batch sizes train slower, but can converge faster.
#Converging meaning getting a stable value.
#32 or 64 is a good starting point

DATA_DIR = '/home/development/Desktop/First GAN Project/GAN Image Generator/GAN-Image-Generator/animal-faces/afhq/dog/'
batch_size = 32
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [12]:
#When arranging the data, the DATA_DIR needs to be the root with the data as sub folders. (ex: I had to create a 'Dog' subdirectory.)
#Tensor has a different meaning in ML versus math.
#Tensors allow for GPU acceleration.
#Tensors hold the data, weights, and biases. They take care of backpropogation with automatic differentiation.
#An image is a 2D array of height x width containing 3 channels for RGB.
#ToTensor normalizes the 3 channels to be within 0-1 for each channel.
#shuffle - Randomize the data being loaded or not
#num_workers - Turn on multi-process data loading with the specified number of loader worker processes
#pin_memory - Enables fast data transfer to CUDA-enabled GPUs

train_ds = ImageFolder(DATA_DIR, transform=T.Compose([
    T.ToTensor(),
    T.Normalize(*stats)]))

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)